In [2]:
import tensorflow as tf
import numpy as np
import sys
import keras
sys.path.append("D:\\documents\\code\\stock\\NNS\\feedback_classifier\\dataset")
sys.path.append("D:\\documents\\code\\stock\\NNS\\inquiryProcessor")
sys.path.append("D:\\documents\\code\\stock\\NNS\\feedback_classifier")
sys.path.append("D:\\documents\\code\\stock\\NNS")
sys.path.append("D:\\documents\\code\\stock")
from model import FeedbackClassifier, FeedbackClassifierBERT, FeedbackClassifierBERTModel
from dataset import FeedbackDataset
from NNS.inquiryProcessor.bert import get_bert_details

In [3]:
def get_ds(BATCH_SIZE: int, dataset_path: str):
    npdataset = FeedbackDataset.get_ds(dataset_path)
    train_examples = tf.convert_to_tensor(npdataset[:, 0], dtype=tf.string)
    train_labels = np.stack(npdataset[:, 1])
    assert len(train_labels.shape) == 1
    train_labels = train_labels.reshape((npdataset[:, 1].shape[0], 1))  # tensorflow can't convert labels easily
    train_labels = tf.convert_to_tensor(train_labels, dtype=tf.int32)
    dataset = tf.data.Dataset.from_tensor_slices((train_examples, train_labels))
    train_ds = dataset.batch(BATCH_SIZE, drop_remainder=False).cache().prefetch(buffer_size=tf.data.AUTOTUNE)
    return train_ds


def train(savepath, epochs=100, model: keras.Model=None):
    # assert(type(savepath) == str and callback == None or type(callback) == keras.callbacks.ModelCheckpoint)
    BATCH_SIZE = 32
    # lets get the dataset first
    train_ds = get_ds(BATCH_SIZE, "D:\\documents\\code\\stock\\NNS\\feedback_classifier\\dataset\\feedback.csv", ) # TODO: TYPE IN YOUR GIT PATH
    print(train_ds)
    # TODO: SOLVE THIS LOCAL PATH PROBLEM
    if model == None:
        tfhub_handle_preprocess, tfhub_handle_encoder = get_bert_details()
        model = FeedbackClassifierBERTModel(tfhub_handle_preprocess, tfhub_handle_encoder)
    analyzer = FeedbackClassifierBERT(model)
    logs = analyzer.train(ds=train_ds, epochs=epochs)
    model.save(savepath, save_format="tf")  # creates a folder inside NNS/InquiryProcessor\
    return logs


def get_model_from_file(filepath, ds: tf.data.Dataset, epochs: int):
    model = keras.models.load_model(filepath, custom_objects={"AdamWeightDecay": FeedbackClassifierBERT.get_optimizer(ds, epochs)})
    return model
    

In [4]:
ds = get_ds(32, "D:\\documents\\code\\stock\\NNS\\feedback_classifier\\dataset\\feedback.csv")
for i in ds.enumerate():
    print(i)

Reading content from D:\documents\code\stock\NNS\feedback_classifier\dataset\feedback.csv


D:\documents\code\stock\NNS\feedback_classifier\dataset\dataset.py:21: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  dataset = pd.read_csv(filepath, warn_bad_lines=True)


(<tf.Tensor: shape=(), dtype=int64, numpy=0>, (<tf.Tensor: shape=(13,), dtype=string, numpy=
array([b"Hey, thank you for the new item. It's lit!!!",
       b'Thank you for the pizza! The quality is above the sky!',
       b'The pizza is lit! Thank a lot!',
       b"Why do you deliver them in plastic packaging! It's bad for the environment! Replace those with organic stuff!",
       b'The pizza stinks as shit! What the hell is this????',
       b'Why the hell did you put the tomatoes inside!? I told you not to put them!',
       b"I've already taken them from the post office! They're so soft and sweet! Thank you and love you!",
       b'I like everything except the price. Could be a bit cheaper. Everything else is great!',
       b'The delivery took a bit long but nothing bad happened then. The shoes are great. Pull them on every day, no problem.',
       b"No comment! They're amazing!", b'Jesus, thank you! So awesome!',
       b'Oh, God, thanks a lot! Ur awesome guys, thank you!!!',
  

In [5]:
savepath = "D:\\documents\\code\\stock\\NNS\\feedback_classifier\\BERT\\feedback_model"
train(savepath)

Reading content from D:\documents\code\stock\NNS\feedback_classifier\dataset\feedback.csv
<PrefetchDataset shapes: ((None,), (None, 1)), types: (tf.string, tf.int32)>
BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


D:\documents\code\stock\NNS\feedback_classifier\dataset\dataset.py:21: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  dataset = pd.read_csv(filepath, warn_bad_lines=True)


Epoch 1/100


ValueError: in user code:

    File "d:\SOFT\anaconda\envs\Stock\lib\site-packages\keras\engine\training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "d:\SOFT\anaconda\envs\Stock\lib\site-packages\keras\engine\training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "d:\SOFT\anaconda\envs\Stock\lib\site-packages\keras\engine\training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "d:\SOFT\anaconda\envs\Stock\lib\site-packages\keras\engine\training.py", line 808, in train_step
        y_pred = self(x, training=True)
    File "d:\SOFT\anaconda\envs\Stock\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None

    ValueError: Exception encountered when calling layer "feedback_classifier_bert_model" (type FeedbackClassifierBERTModel).
    
    in user code:
    
       